In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
%matplotlib inline
import seaborn as sns
import matplotlib.pylab as plt

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from DSI_Capstone_Steemit.utils.utils import(
    load_data_and_description,
    
)




In [ ]:
data,feature_names,data_desc = load_data_and_description(data_type='posts_tfidf')

In [ ]:
import scipy.sparse as ssp


train_features = data_desc.ix[:,['number of body tags',
                                   'number of body urls',
                                   'number of image urls',
                                   'number of body mentions',
                                   'number of image urls',
                                   'number of youtube urls',
                                   'language',
                                   'author_reputation_scaled',
                                   'number of steem counts']]




In [ ]:
data_desc['number of steem counts'] = data_desc['number of steem counts'].fillna(0)

In [ ]:
train = pd.get_dummies(train_features)

num_image_urls = train['number of image urls'].values[:,0]
train.fillna(0).drop('number of image urls',axis = 1, inplace=True)

train['number of image urls'] = num_image_urls
train_sparse = ssp.csr_matrix(train)



In [ ]:

new_data = ssp.hstack([data,train_sparse])
train = new_data.tocsr()

In [ ]:
new_data

In [ ]:
from sklearn.cluster import MeanShift,AgglomerativeClustering
from sklearn.decomposition import PCA,TruncatedSVD

In [ ]:
pca = TruncatedSVD(n_components=3)

In [ ]:
pca_output = pca.fit_transform(data)

In [ ]:
data_desc['pca1'] = pca_output[:,0]
data_desc['pca2'] = pca_output[:,1]

In [ ]:
top_categories = data_desc['category'].value_counts()[data_desc['category'].value_counts() > 200].index

In [ ]:
len(data_desc['language'].unique())

In [ ]:
df_plot = data_desc.ix[data_desc['category'].isin(top_categories),'category']



In [ ]:
fig,ax = plt.subplots(1,1,figsize = (10,10))
df_plot = data_desc.ix[data_desc['category'].isin(top_categories),'category']
sns.countplot(y = df_plot,ax = ax,order=df_plot.value_counts().index)

In [ ]:
hue = 'category'
unique_values = np.sort(data_desc[hue].unique())
import colorsys
N = len(unique_values)
HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in range(N)]
RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)


for i,value in enumerate(unique_values):
    short = data_desc[data_desc[hue] == value]
    
    x = short['pca1']
    y = short['pca2']
    plt.plot(x,y, label = hue, color = RGB_tuples[i])

plt.legend()

In [ ]:
x

In [ ]:
unique_values

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize = (10,5))
x = pca_output[:,0]
y = pca_output[:,1]
ax1.scatter(x,y)

x = pca_output[:,0]
y = pca_output[:,2]

ax2.scatter(x,y)


In [ ]:
fig,(ax1) = plt.subplots(1,1, figsize = (5,5))
x = pca_output[:,0]
y = pca_output[:,1]
ax1.scatter(x,y)

ax1.set_title('1st and 2nd SVD Components (Similar to PCA)')